<a href="https://colab.research.google.com/github/Giuse1093/CSI_Project4/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import ast
from scipy.fft import fft
from sklearn.preprocessing import StandardScaler
import warnings

# Ignoriamo warning minori di pandas
warnings.filterwarnings('ignore')

# Colonne contenenti le serie temporali
TS_COLS = ['hr_time_series', 'resp_time_series', 'stress_time_series']

def clean_series_interpolation(series_str):
    """
    Replica la logica di 'Accelerometer_Preprocessing.ipynb':
    1. Parsing da stringa a lista.
    2. Sostituzione valori errati (<=0) con NaN.
    3. Interpolazione Lineare per riempire i buchi mantenendo il trend temporale.
    """
    if not isinstance(series_str, str):
        return np.array([])

    try:
        # 1. Parsing
        data = np.array(ast.literal_eval(series_str), dtype=float)

        # 2. Rimuovi outlier/errori sensore (-1, -2, 0) impostandoli a NaN
        data[data <= 0] = np.nan

        # Trasforma in Series per usare interpolate
        s_data = pd.Series(data)

        # 3. Interpolazione (Linear) - Fondamentale per non perdere la sequenza
        s_data = s_data.interpolate(method='linear', limit_direction='both')

        # Se restano NaN (es. serie vuota), riempi con 0
        s_data = s_data.fillna(0)

        return s_data.values
    except:
        return np.array([])

def extract_features_datasets_py(clean_data):
    """
    Replica ESATTA di 'datasets.py' -> 'calculate_params':
    Estrae: Mean, Std, Percentili (25, 50, 75), Energia Spettrale.
    """
    if len(clean_data) == 0:
        return [0.0] * 6

    # Statistiche base
    mean = np.mean(clean_data)
    std = np.std(clean_data)
    p25 = np.percentile(clean_data, 25)
    p50 = np.percentile(clean_data, 50) # Mediana
    p75 = np.percentile(clean_data, 75)

    # Energia Spettrale (FFT) - Come nel file della prof
    f_transform = fft(clean_data)
    spectral_energy = np.sum(np.abs(f_transform)**2) / len(clean_data)

    return [mean, std, p25, p50, p75, spectral_energy]

def process_dataframe(df):
    """
    Prende il dataframe grezzo e restituisce un dataframe di feature numeriche.
    """
    extracted_features = []

    for _, row in df.iterrows():
        row_feats = []
        for col in TS_COLS:
            # A. Pulizia
            clean_seq = clean_series_interpolation(row[col])
            # B. Estrazione Feature
            feats = extract_features_datasets_py(clean_seq)
            row_feats.extend(feats)

        extracted_features.append(row_feats)

    # Creiamo nomi colonne parlanti
    col_names = []
    prefixes = ['hr', 'resp', 'stress']
    suffixes = ['mean', 'std', 'p25', 'p50', 'p75', 'energy']
    for p in prefixes:
        for s in suffixes:
            col_names.append(f"{p}_{s}")

    return pd.DataFrame(extracted_features, columns=col_names)

print("Funzioni di preprocessing definite correttamente.")

Funzioni di preprocessing definite correttamente.


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

def get_model(input_dim):
    """
    Crea una rete neurale densa per regressione.
    """
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),

        # Primo layer ampio
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3), # Dropout per evitare overfitting (visto che i dati sono pochi per client)

        # Secondo layer
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),

        # Terzo layer
        layers.Dense(32, activation='relu'),

        # Output Layer (1 neurone lineare per predire 0-100)
        layers.Dense(1, activation='linear')
    ])

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

print("Modello definito.")

Modello definito.


In [3]:
# --- CONFIGURAZIONE ---
TRAIN_ROOT = "CSV_train"
ROUNDS = 15
EPOCHS_PER_CLIENT = 3
BATCH_SIZE = 16

def run_federated_learning_grouped():
    # 1. Identificazione dei Client (Sottocartelle)
    # Cerca tutte le cartelle dentro CSV_train (es. CSV_train/Group_1)
    # Se CSV_train contiene direttamente i csv, adattiamo la logica,
    # ma assumiamo la struttura a sottocartelle descritta.

    # Trova le sottocartelle
    subfolders = [f.path for f in os.scandir(TRAIN_ROOT) if f.is_dir()]

    # Se non ci sono sottocartelle, prova a vedere se c'è un livello intermedio
    # (A volte gli zip creano CSV_train/CSV_train/...)
    if not subfolders:
        # Fallback: trattiamo la cartella root come un unico client se non ci sono gruppi
        # Ma l'obiettivo è avere più gruppi.
        print("Nessuna sottocartella trovata direttamente. Controllo ricorsivo...")
        # Logica semplice: ogni cartella che contiene csv è un client
        client_folders = []
        for root, dirs, files in os.walk(TRAIN_ROOT):
            csv_files = [f for f in files if f.endswith('.csv')]
            if len(csv_files) > 0:
                client_folders.append(root)
    else:
        client_folders = subfolders

    print(f"Trovati {len(client_folders)} Client (Gruppi):")
    for c in client_folders: print(f" - {os.path.basename(c)}")

    if not client_folders:
        print("ERRORE CRITICO: Nessun dato trovato.")
        return None, None

    # 2. Inizializzazione Modello Globale
    # Leggiamo un file a caso per le dimensioni
    first_csv = [f for f in os.listdir(client_folders[0]) if f.endswith('.csv')][0]
    sample_df = pd.read_csv(os.path.join(client_folders[0], first_csv), sep=';')
    sample_X = process_dataframe(sample_df.head(5))
    input_dim = sample_X.shape[1]

    global_model = get_model(input_dim)
    global_weights = global_model.get_weights()

    # --- LOOP FEDERATO ---
    for round_num in range(ROUNDS):
        print(f"\n--- Round {round_num + 1}/{ROUNDS} ---")
        local_weights_list = []
        local_losses = []

        for client_folder in client_folders:
            try:
                # A. AGGREGAZIONE DATI DEL GRUPPO (CLIENT)
                # Uniamo tutti i CSV di questo gruppo in un unico DataFrame
                group_csvs = [os.path.join(client_folder, f) for f in os.listdir(client_folder) if f.endswith('.csv')]

                df_list = []
                for csv_file in group_csvs:
                    df_temp = pd.read_csv(csv_file, sep=';')
                    df_list.append(df_temp)

                if not df_list: continue

                df_client = pd.concat(df_list, ignore_index=True)

                # B. PREPROCESSING
                X_client_extracted = process_dataframe(df_client)
                y_client = df_client['label'].values

                # C. SPLIT TRAIN / VALIDATION (Locale al client)
                # Parte dei dati usata per il training, parte per validare
                X_train, X_val, y_train, y_val = train_test_split(X_client_extracted, y_client, test_size=0.2, random_state=42)

                # D. NORMALIZZAZIONE (StandardScaler Locale)
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(X_train)
                X_val_scaled = scaler.transform(X_val) # Usa lo scaler del train

                # E. TRAINING LOCALE
                client_model = get_model(input_dim)
                client_model.set_weights(global_weights)

                # Early Stopping sulla Validation Loss locale
                es = callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

                history = client_model.fit(
                    X_train_scaled, y_train,
                    validation_data=(X_val_scaled, y_val),
                    epochs=EPOCHS_PER_CLIENT,
                    batch_size=BATCH_SIZE,
                    verbose=0,
                    callbacks=[es]
                )

                # Salviamo i pesi e la loss finale
                local_weights_list.append(client_model.get_weights())
                val_loss = history.history['val_loss'][-1]
                local_losses.append(val_loss)

                # print(f"   Client {os.path.basename(client_folder)} -> Val Loss: {val_loss:.2f}")

            except Exception as e:
                print(f"   Errore Client {os.path.basename(client_folder)}: {e}")

        # F. AGGREGAZIONE (FedAvg)
        if local_weights_list:
            avg_weights = []
            for layer_idx in range(len(global_weights)):
                layer_weights = [w[layer_idx] for w in local_weights_list]
                avg_weights.append(np.mean(layer_weights, axis=0))

            global_weights = avg_weights
            global_model.set_weights(global_weights)

            avg_loss = np.mean(local_losses)
            print(f"   >>> Round completato. Avg Global Validation Loss: {avg_loss:.2f}")

    return global_model, input_dim

# AVVIO
final_model, feature_dim = run_federated_learning_grouped()

NameError: name 'os' is not defined

In [ ]:
if final_model:
    print("\nGenerazione predizioni per x_test.csv...")
    df_test = pd.read_csv("x_test.csv", sep=';')

    # Feature Extraction
    X_test_extracted = process_dataframe(df_test)

    # Normalizzazione (Fit sul test set per adattamento dominio semplice)
    scaler_test = StandardScaler()
    X_test_scaled = scaler_test.fit_transform(X_test_extracted)

    # Predizione
    predictions = final_model.predict(X_test_scaled)

    # Creazione CSV
    submission = pd.DataFrame({
        'id': df_test['id'],
        'label': predictions.flatten()
    })

    submission.to_csv('submission.csv', index=False)
    print("File 'submission.csv' pronto per il download!")
    print(submission.head())
else:
    print("Training fallito, nessun modello generato.")